In [1]:
import numpy as np
import pandas as pd
import re
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time # for sleep time
import csv
import functools
import datetime #for csv file title

In [3]:
now = datetime.datetime.now() #for csv file title
folder = 'airBnb_' + str(now.year) + '-' + str(now.month) + '-' + str(now.day)
pathsalida ='C:/Users/marti/Dev/Airbnb/' + folder
pathsalida
try:
    os.makedirs(pathsalida, 0o777)
except:
    print("Ya existe")

Ya existe


In [4]:
driver = webdriver.Chrome(r'chromedriver.exe')

CP=["Big Bear", "Whistler", "Purgatory Resort"]
files_names = []
for j in range (0, len(CP)):
    driver.get('https://www.airbnb.mx/s/homes?refinement_paths%5B%5D=%2Fhomes&search_type=section_navigation&s_tag=yVNy3gQo')
    time.sleep(10)
    element = driver.find_element_by_id("Koan-via-SearchHeader__input")
    city = CP[j]
    element.send_keys(city)
    time.sleep(10)
    element.send_keys(Keys.RETURN)
    time.sleep(10)
    
    now = datetime.datetime.now() #for csv file title

    #For csv file names
    def reduce_concat(x, sep=""):
        return functools.reduce(lambda x, y: str(x) + sep + str(y), x)

    def paste(*lists, sep=" ", collapse=None):
        result = map(lambda x: reduce_concat(x, sep=sep), zip(*lists))
        if collapse is not None:
            return reduce_concat(result, sep=collapse)
        return list(result)

    #Open CSV Writer
    os.chdir(pathsalida)
    filename = paste(['airBnb_'],[city], [now.year], ['-'], [now.month], ['-'], [now.day],['.csv'], sep='')
    filename = ''.join(filename)
    files_names.append(filename)
    csv_file = open(filename, 'w', encoding='utf-8',newline = '')
    writer = csv.writer(csv_file)

    index = 1
    i = 1

    house_all_links = []

    while True:
        driver.maximize_window()
        if index > 17:
            break
        try:
            print("Getting list of links from Page " + str(index))
            index = index + 1
            
            time.sleep(30) # give new page time to load

            house_list = (driver.find_elements_by_xpath('//a[@class="_1ol0z3h"]'))

            house_links = [house.get_attribute('href') for house in house_list ] 

            house_all_links.extend(house_links)

            # Locate the next button on the page.
            wait_button = WebDriverWait(driver, 20)

            next_button = wait_button.until(EC.element_to_be_clickable((By.XPATH,'//li[@class="_r4n1gzb"]')))

            next_button.click()


        except Exception as e:
            print(e)
            continue
    """
    airbnb_city = []
    airbnb_house_title = []
    airbnb_house_type = []
    airbnb_price = []
    airbnb_user = []
    airbnb_no_of_reviews = []
    airbnb_rating = []
    airbnb_house_detail = []
    airbnd_url = []
    """
    print(house_all_links)
    for url in house_all_links:

        # Initialize an empty dictionary for each review
        airbnb_dict = {}

        driver.get(url)

        print(url)
        try:
            time.sleep(3)
            try:
                house_title = driver.find_element_by_xpath('//h1/span[@class="_18hrqvin"]').text
            except:
                print("issue in house title")
                house_title = None

            try:
                house_type =  driver.find_element_by_xpath('//div[@class="_1p3joamp"]').text
            except:
                print("issue in house type")
                house_type = None

            time.sleep(5)    

            try:

                price = driver.find_element_by_xpath('//span/span[@class="_doc79r"]').text
            except:
                print("issue in price")
                price = None        

            try:
                user = driver.find_element_by_xpath('//div[@class="_8b6uza1"]').text
            except:
                print("issue in user")
                user = None  

            try:
                no_of_ratings = driver.find_element_by_xpath('//div[@class="_vy3ibx"]/h2/span').text

            except:
                print("issue in no_of_ratings")
                no_of_ratings = None  

            time.sleep(3.5)    
            try:
                rating = driver.find_element_by_xpath('//div[@class="_l0ao8q"]//div[@itemprop="ratingValue"]').get_attribute('content')

            except:
                print("issue in rating")
                rating = None

            time.sleep(2)        
            try:
                max_occupancy = driver.find_elements_by_xpath('//div/div[@class="_36rlri"]/div[@class="_czm8crp"]')

                house_detail = [detail.text for detail in max_occupancy ]

            except:
                print("Issue in occupncy")
                house_detail = None

            
            
            airbnb_dict['city'] = city
            airbnb_dict['house_title'] = house_title
            airbnb_dict['house_type'] = house_type
            airbnb_dict['price'] = price
            airbnb_dict['user'] = user
            airbnb_dict['no_of_reviews'] = no_of_ratings
            airbnb_dict['rating'] = rating
            airbnb_dict['house_detail'] = house_detail
            airbnb_dict['url'] = url
            
            """
            airbnb_city.append(city)
            airbnb_house_title.append(house_title)
            airbnb_house_type.append(house_type)
            airbnb_price.append(price)
            airbnb_user.append(user)
            airbnb_no_of_reviews.append(no_of_ratings)
            airbnb_rating.append(rating)
            airbnb_house_detail.append(house_detail)
            airbnb_url.append(url)
            """

            print("writing")
            writer.writerow(airbnb_dict.values())
            

            print("House number: ", i, " of ", len(house_all_links))
            print(house_title)
            print(house_type)
            print(price)
            print(user)
            print(no_of_ratings)
            print(rating)
            print(house_detail)
            i+= 1

        except Exception as e:
            print(e)
            continue
            
    """
    airbnb_dict = pd.DataFrame(columns=["city","house_title","house_type","price","owner_name","no_of_reviews","rating","occupancy", "url"])
    airbnb_dict['city'] = airbnb_city
    airbnb_dict['house_title'] = airbnb_house_title
    airbnb_dict['house_type'] = airbnb_house_type
    airbnb_dict['price'] = airbnb_price
    airbnb_dict['user'] = airbnb_user
    airbnb_dict['no_of_reviews'] = airbnb_no_of_reviews
    airbnb_dict['rating'] = airbnb_rating
    airbnb_dict['house_detail'] = airbnb_house_detail
    airbnb_dict['url'] = airbnd_url
    """
    csv_file.close()
    #driver.close()
driver.close()

Getting list of links from Page 1
Getting list of links from Page 2
Getting list of links from Page 3
Getting list of links from Page 4
Getting list of links from Page 5
Getting list of links from Page 6
Getting list of links from Page 7
Getting list of links from Page 8
Getting list of links from Page 9
Getting list of links from Page 10
Getting list of links from Page 11
Getting list of links from Page 12
Getting list of links from Page 13
Getting list of links from Page 14
Getting list of links from Page 15
Getting list of links from Page 16
Getting list of links from Page 17
['https://www.airbnb.mx/rooms/11820466?location=Big%20Bear&previous_page_section_name=1000', 'https://www.airbnb.mx/rooms/3344360?location=Big%20Bear&previous_page_section_name=1000', 'https://www.airbnb.mx/rooms/32370417?location=Big%20Bear&previous_page_section_name=1000', 'https://www.airbnb.mx/rooms/22530081?location=Big%20Bear&previous_page_section_name=1000', 'https://www.airbnb.mx/rooms/27979739?location

https://www.airbnb.mx/rooms/11820466?location=Big%20Bear&previous_page_section_name=1000
writing
House number:  1  of  306
Serenity in the Tree Tops
Cabaña entera
$864
Jim & Laura
238 evaluaciones
5
['3 huéspedes', 'estudio', '2 camas', '1 baño', '\U000f0004\n238 evaluaciones', '\U000f0019\nVerificado']
https://www.airbnb.mx/rooms/3344360?location=Big%20Bear&previous_page_section_name=1000
writing
House number:  2  of  306
Stay 7-11!Hot tub,WiFi,grill,no pets allowed!
Cabaña entera
$1,920
Richard And Alex
273 evaluaciones
5
['6 huéspedes', '2 recámaras', '3 camas', '1 baño', '\U000f0004\n273 evaluaciones', '\U000f0019\nVerificado']
https://www.airbnb.mx/rooms/32370417?location=Big%20Bear&previous_page_section_name=1000
writing
House number:  3  of  306
Unique and Modern A-Frame
Cabaña entera
$1,114
Scott
31 evaluaciones
5
['4 huéspedes', '1 recámara', '4 camas', '1 baño completo y uno de tocador', '\U000f0004\n172 evaluaciones', '\U000f0019\nVerificado']
https://www.airbnb.mx/rooms/225

https://www.airbnb.mx/rooms/18684079?location=Big%20Bear&previous_page_section_name=1000
writing
House number:  26  of  306
Cathy Cottage 6 - King w/Spa & Fireplace
Casa entera
$768
Sleepy Forest Resort
7 evaluaciones
5
['2 huéspedes', 'estudio', '1 cama', '1 baño', '\U000f0004\n302 evaluaciones', '\U000f0019\nVerificado']
https://www.airbnb.mx/rooms/10944217?location=Big%20Bear&previous_page_section_name=1000
writing
House number:  27  of  306
Romantic Guest House, Close to Village & Lake
Casa de huéspedes completa
$1,901
Dave & Heidi
188 evaluaciones
5
['2 huéspedes', '1 recámara', '2 camas', '1 baño', '\U000f0004\n188 evaluaciones', '\U000f0019\nVerificado']
https://www.airbnb.mx/rooms/18684077?location=Big%20Bear&previous_page_section_name=1000
writing
House number:  28  of  306
Cathy Cottage 4 - King w/Spa & Fireplace
Habitación en hotel
$768
Sleepy Forest Resort
3 evaluaciones
5
['2 huéspedes', '1 recámara', '1 cama', '1 baño privado', '\U000f0004\n302 evaluaciones', '\U000f0019\

writing
House number:  51  of  306
"88 Acres Lodge" W/Hot Tub!
Cabaña entera
$1,824
Matt
336 evaluaciones
5
['4 huéspedes', '2 recámaras', '2 camas', '1 baño', '\U000f0004\n2238 evaluaciones', '\U000f1101\n1 Recomendación', '\U000f0019\nVerificado']
https://www.airbnb.mx/rooms/2571943?location=Big%20Bear&previous_page_section_name=1000
writing
House number:  52  of  306
Retro cabin with a view!
Cabaña entera
$941
Chelsea
147 evaluaciones
5
['4 huéspedes', '1 recámara', '2 camas', '1 baño', '\U000f0004\n173 evaluaciones', '\U000f0019\nVerificado']
https://www.airbnb.mx/rooms/22573442?location=Big%20Bear&previous_page_section_name=1000
writing
House number:  53  of  306
Cozy Big Bear Cabin, Minutes from the lake!
Cabaña entera
$1,267
Justin & Forough
155 evaluaciones
5
['6 huéspedes', '2 recámaras', '2 camas', '1 baño', '\U000f0004\n207 evaluaciones', '\U000f1101\n1 Recomendación', '\U000f0019\nVerificado']
https://www.airbnb.mx/rooms/8614041?location=Big%20Bear&previous_page_section_nam

issue in rating
writing
House number:  76  of  306
THE CABIN LIFE IS CALLING YOU! 2BD/1BA W/ JACUZZI
Cabaña entera
$1,498
Ashlliegh
1 evaluación
None
['6 huéspedes', '2 recámaras', '3 camas', '1 baño', '\U000f0004\n1 evaluación', '\U000f0019\nVerificado']
https://www.airbnb.mx/rooms/34359282?location=Big%20Bear&previous_page_section_name=1000
writing
House number:  77  of  306
Cabin in the Pines
Cabaña entera
$1,133
Tristen
7 evaluaciones
5
['6 huéspedes', '2 recámaras', '5 camas', '1 baño completo y uno de tocador', '\U000f0004\n21 evaluaciones', '\U000f0019\nVerificado']
https://www.airbnb.mx/rooms/35866642?location=Big%20Bear&previous_page_section_name=1000
writing
House number:  78  of  306
Simple, Clean, Affordable
Casa entera
$1,344
Dawn
8 evaluaciones
5
['5 huéspedes', '3 recámaras', '4 camas', '2 baños', '\U000f0004\n8 evaluaciones', '\U000f0019\nVerificado']
https://www.airbnb.mx/rooms/36155074?location=Big%20Bear&previous_page_section_name=1000
writing
House number:  79  of  

...

issue in rating
writing
House number:  189  of  290
Ski-in/out condo w/ shared pool, hot tubs, fitness room, private balcony, & view
Condominio completo
$3,111
Vacasa Colorado
1 evaluación
None
['6 huéspedes', '1 recámara', '3 camas', '1 baño', '\U000f0004\n1238 evaluaciones']
https://www.airbnb.mx/rooms/35262364?location=Purgatory%20Resort&previous_page_section_name=1000
issue in rating
writing
House number:  190  of  290
Updated, mountain condo w/ a balcony & shared hot tub - close to resort!
Condominio completo
$3,226
Vacasa Colorado
1 evaluación
None
['6 huéspedes', '2 recámaras', '4 camas', '2 baños', '\U000f0004\n1238 evaluaciones']
https://www.airbnb.mx/rooms/16252869?location=Purgatory%20Resort&previous_page_section_name=1000
issue in rating
writing
House number:  191  of  290
Mountain Townhome - Great Views - Free on Demand Ski Shuttle
Casa adosada completa
$3,533
Premier Vacation Rentals
1 evaluación
None
['8 huéspedes', '3 recámaras', '4 camas', '3 baños', '\U000f0004\n457 e

https://www.airbnb.mx/rooms/16252830?location=Purgatory%20Resort&previous_page_section_name=1000
issue in rating
writing
House number:  212  of  290
Luxury Townhome - Great Views/Fire Pit/BBQ - 4th Night Free - Heated Pool
Casa adosada completa
$3,860
Premier Vacation Rentals
2 evaluaciones
None
['10 huéspedes', '3 recámaras', '5 camas', '3 baños', '\U000f0004\n457 evaluaciones']
https://www.airbnb.mx/rooms/16992411?location=Purgatory%20Resort&previous_page_section_name=1000
writing
House number:  213  of  290
Slopeside Studio-Great Value!
Condominio completo
$1,536
Rick
30 evaluaciones
5
['4 huéspedes', 'estudio', '1 cama', '1 baño', '\U000f0004\n50 evaluaciones', '\U000f0019\nVerificado']
https://www.airbnb.mx/rooms/16253089?location=Purgatory%20Resort&previous_page_section_name=1000
writing
House number:  214  of  290
Ski in/Ski Out - Great Views - Deck - Affordable
Departamento entero
$1,440
Premier Vacation Rentals
6 evaluaciones
5
['4 huéspedes', 'estudio', '2 camas', '1 baño', '

writing
House number:  235  of  290
Studio Condo at Purgatory Resort 03
Condominio completo
$2,324
Vacation Accommodations
7 evaluaciones
5
['4 huéspedes', 'estudio', '3 camas', '1 baño', '\U000f0004\n234 evaluaciones', '\U000f0019\nVerificado']
https://www.airbnb.mx/rooms/29678305?location=Purgatory%20Resort&previous_page_section_name=1000
issue in rating
writing
House number:  236  of  290
Luxury Ski In/Out townhome on Creek - Views - End Unit - Closest to slopes
Casa adosada completa
$3,553
Premier Vacation Rentals
2 evaluaciones
None
['10 huéspedes', '3 recámaras', '5 camas', '4 baños', '\U000f0004\n457 evaluaciones']
https://www.airbnb.mx/rooms/29365619?location=Purgatory%20Resort&previous_page_section_name=1000
issue in rating
writing
House number:  237  of  290
Ski/In-Ski/Out @ Purgatory Ski Area TVC B-1
Condominio completo
$3,457
Chris
1 evaluación
None
['6 huéspedes', '2 recámaras', '5 camas', '2 baños', '\U000f0004\n42 evaluaciones']
https://www.airbnb.mx/rooms/29158420?locat

https://www.airbnb.mx/rooms/16270886?location=Purgatory%20Resort&previous_page_section_name=1000
issue in rating
writing
House number:  259  of  290
Blackbear Cabin in Purgatory Ski resort
Casa adosada completa
$3,841
Siri
3 evaluaciones
None
['8 huéspedes', '3 recámaras', '5 camas', '2 baños completos y uno de tocador', '\U000f0004\n9 evaluaciones', '\U000f0019\nVerificado']
https://www.airbnb.mx/rooms/29800546?location=Purgatory%20Resort&previous_page_section_name=1000
issue in rating
writing
House number:  260  of  290
Ski/In-Ski/Out @ Purgatory Ski Area TVC D-2
Condominio completo
$2,977
Chris
1 evaluación
None
['6 huéspedes', '2 recámaras', '4 camas', '2 baños', '\U000f0004\n42 evaluaciones']
https://www.airbnb.mx/rooms/16887651?location=Purgatory%20Resort&previous_page_section_name=1000
writing
House number:  261  of  290
Skier's dream w/ski-in/out access, hot tub, pool, & more!
Condominio completo
$2,573
Vacasa Colorado
4 evaluaciones
4
['6 huéspedes', '2 recámaras', '3 camas', 

https://www.airbnb.mx/rooms/19843688?location=Purgatory%20Resort&previous_page_section_name=1000
issue in rating
writing
House number:  283  of  290
Two Bedroom Condo Walk to Ski Lift PTH 112
Condominio completo
$3,361
Vacation Accommodations
1 evaluación
None
['6 huéspedes', '2 recámaras', '3 camas', '2 baños', '\U000f0004\n234 evaluaciones', '\U000f0019\nVerificado']
https://www.airbnb.mx/rooms/16253052?location=Purgatory%20Resort&previous_page_section_name=1000
issue in no_of_ratings
issue in rating
writing
House number:  284  of  290
Premier Building at Purgatory - Views - Ski in/Ski Out - Corner Unit
Departamento entero
$5,089
Premier Vacation Rentals
None
None
['10 huéspedes', '4 recámaras', '5 camas', '4 baños', '\U000f0004\n457 evaluaciones']
https://www.airbnb.mx/rooms/23147973?location=Purgatory%20Resort&previous_page_section_name=1000
issue in rating
writing
House number:  285  of  290
This IS the location on the mountain! Stunning 4-bedroom, 3.5 bath condo
Condominio comple

In [10]:
cols =["city","house_title","house_type","price","owner_name","no_of_reviews","rating","occupancy", "url"]

#read in the csv file
files_names
#airbnbdf = pd.read_csv(filename,encoding = 'utf-8', names=cols)
#airbnbdf.head(5)

,city,house_title,house_type,price,owner_name,no_of_reviews,rating,occupancy,url
0,Whistler,˙Studio Condo Whistler Village 223,Departamento entero,"$1,086",Bill,398 evaluaciones,5.0,"['2 huéspedes', 'estudio', '1 cama', '1 baño',...",https://www.airbnb.mx/rooms/12071124?location=...
1,Whistler,Studio Condo Upper Village Tranquility,Departamento entero,"$1,158",Bill,478 evaluaciones,5.0,"['4 huéspedes', 'estudio', '2 camas', '1 baño'...",https://www.airbnb.mx/rooms/7309668?location=W...
2,Whistler,˯Studio Condo Whistler Village 422,Departamento entero,"$1,158",Bill,NaN,5.0,"['3 huéspedes', 'estudio', '2 camas', '1 baño'...",https://www.airbnb.mx/rooms/12310742?location=...
3,Whistler,Peak View Penthouse,Loft entero,"$1,868",Jane & James,NaN,5.0,"['4 huéspedes', '1 recámara', '1 cama', '1 bañ...",https://www.airbnb.mx/rooms/13348663?location=...
4,Whistler,1 BR Townhome with Private Hot Tub - FREE PARKING,Casa adosada completa,"$1,665",Ian,312 evaluaciones,4.5,"['4 huéspedes', '1 recámara', '2 camas', '1 ba...",https://www.airbnb.mx/rooms/2520507?location=W...


In [11]:
#verify the datatypes
airbnbdf.dtypes

city              object
house_title       object
house_type        object
price             object
owner_name        object
no_of_reviews     object
rating           float64
occupancy         object
url               object
dtype: object

In [12]:
airbnbdf.price = airbnbdf.price.astype(str)
airbnbdf.no_of_reviews = airbnbdf.no_of_reviews.astype(str)


def remove_special_chars(val):
    new_val = re.findall('[0-9]',val)
    return "".join(new_val)

airbnbdf.price = airbnbdf.price.apply(remove_special_chars)
print(airbnbdf.price.head(5))


0    1086
1    1158
2    1158
3    1868
4    1665
Name: price, dtype: object


In [13]:

airbnbdf.no_of_reviews = airbnbdf.no_of_reviews.apply(remove_special_chars)
print(airbnbdf.no_of_reviews.head(5))


0    398
1    478
2       
3       
4    312
Name: no_of_reviews, dtype: object


In [14]:
def split_to_list(str_val):
    return str_val.split(",")
    

airbnbdf['occupancy_lst'] = airbnbdf.occupancy.apply(split_to_list)

airbnbdf.occupancy_lst.sample(5)

6     [['4 huéspedes',  '1 recámara',  '1 cama',  '1...
16    [['4 huéspedes',  '1 recámara',  '1 cama',  '1...
12    [['4 huéspedes',  '1 recámara',  '2 camas',  '...
7     [['2 huéspedes',  '1 cama',  '1 baño',  '\U000...
15    [['4 huéspedes',  '1 recámara',  '2 camas',  '...
Name: occupancy_lst, dtype: object

In [15]:
len(airbnbdf)

airbnbdf.occupancy_lst[0][0]


"['2 huéspedes'"

In [16]:
airbnbdf = airbnbdf.assign(no_of_guest=airbnbdf.price.mean(), house_size=airbnbdf.price.mean(), no_of_beds = airbnbdf.price.mean(), no_of_baths=airbnbdf.price.mean())

for i in range(len(airbnbdf)):
    try:

        airbnbdf.no_of_guest[i] = airbnbdf.occupancy_lst[i][0]
        airbnbdf.house_size[i]  = airbnbdf.occupancy_lst[i][1]
        airbnbdf.no_of_beds[i]  = airbnbdf.occupancy_lst[i][2]
        airbnbdf.no_of_baths[i]  = airbnbdf.occupancy_lst[i][3]
        
    except:
        continue

In [17]:
airbnbdf.no_of_guest = airbnbdf.no_of_guest.apply(remove_special_chars)
print(airbnbdf.no_of_guest.head(5))

0    2
1    4
2    3
3    4
4    4
Name: no_of_guest, dtype: object


In [18]:
print(sum(airbnbdf.house_size.isna()))

print(sum(airbnbdf.house_size.isnull()))

0
0


In [19]:
def convert_chars(val):
   ##estudio cuenta como 0.5 de un cuarto
    if val == " 'estudio'":
        new_val = '0.5'
    else:
        new_val = re.findall('[0-9]',val)
    
    return "".join(new_val)

airbnbdf.house_size = airbnbdf.house_size.astype(str)
airbnbdf = airbnbdf.assign(rooms=airbnbdf.house_size.apply(convert_chars))

airbnbdf.sample(14)

,city,house_title,house_type,price,owner_name,no_of_reviews,rating,occupancy,url,occupancy_lst,no_of_guest,house_size,no_of_beds,no_of_baths,rooms
14,Whistler,Symphony Studio in Whistler Village,Departamento entero,1737,Musubi,198,5.0,"['4 huéspedes', 'estudio', '2 camas', '1 baño'...",https://www.airbnb.mx/rooms/18074253?location=...,"[['4 huéspedes', 'estudio', '2 camas', '1 b...",4,'estudio','2 camas','1 baño',0.5
2,Whistler,˯Studio Condo Whistler Village 422,Departamento entero,1158,Bill,,5.0,"['3 huéspedes', 'estudio', '2 camas', '1 baño'...",https://www.airbnb.mx/rooms/12310742?location=...,"[['3 huéspedes', 'estudio', '2 camas', '1 b...",3,'estudio','2 camas','1 baño',0.5
16,Whistler,Minimalist Vibe with Amazing Mountain Views,Departamento con servicios completo,1868,Victoria And Christian,337,5.0,"['4 huéspedes', '1 recámara', '1 cama', '1 bañ...",https://www.airbnb.mx/rooms/13283018?location=...,"[['4 huéspedes', '1 recámara', '1 cama', '1...",4,'1 recámara','1 cama','1 baño',1
3,Whistler,Peak View Penthouse,Loft entero,1868,Jane & James,,5.0,"['4 huéspedes', '1 recámara', '1 cama', '1 bañ...",https://www.airbnb.mx/rooms/13348663?location=...,"[['4 huéspedes', '1 recámara', '1 cama', '1...",4,'1 recámara','1 cama','1 baño',1
17,Whistler,Cozy Whistler Village Studio,Condominio completo,1433,Tim And Andrea,312,5.0,"['4 huéspedes', 'estudio', '2 camas', '1 baño'...",https://www.airbnb.mx/rooms/4269069?location=W...,"[['4 huéspedes', 'estudio', '2 camas', '1 b...",4,'estudio','2 camas','1 baño',0.5
11,Whistler,˙Studio Condo Whistler Village 207,Departamento entero,1520,Bill,286,5.0,"['4 huéspedes', 'estudio', '2 camas', '1 baño'...",https://www.airbnb.mx/rooms/16041781?location=...,"[['4 huéspedes', 'estudio', '2 camas', '1 b...",4,'estudio','2 camas','1 baño',0.5
15,Whistler,Ski in/out Stylish condo with pool/hot tub & v...,Departamento entero,1289,Annika,203,5.0,"['4 huéspedes', '1 recámara', '2 camas', '1 ba...",https://www.airbnb.mx/rooms/4249398?location=W...,"[['4 huéspedes', '1 recámara', '2 camas', '...",4,'1 recámara','2 camas','1 baño',1
10,Whistler,1 BR Townhouse with Private Hot Tub - FREE PAR...,Casa adosada completa,1839,Ian,318,4.5,"['5 huéspedes', '1 recámara', '3 camas', '1 ba...",https://www.airbnb.mx/rooms/4828768?location=W...,"[['5 huéspedes', '1 recámara', '3 camas', '...",5,'1 recámara','3 camas','1 baño',1
1,Whistler,Studio Condo Upper Village Tranquility,Departamento entero,1158,Bill,478,5.0,"['4 huéspedes', 'estudio', '2 camas', '1 baño'...",https://www.airbnb.mx/rooms/7309668?location=W...,"[['4 huéspedes', 'estudio', '2 camas', '1 b...",4,'estudio','2 camas','1 baño',0.5
4,Whistler,1 BR Townhome with Private Hot Tub - FREE PARKING,Casa adosada completa,1665,Ian,312,4.5,"['4 huéspedes', '1 recámara', '2 camas', '1 ba...",https://www.airbnb.mx/rooms/2520507?location=W...,"[['4 huéspedes', '1 recámara', '2 camas', '...",4,'1 recámara','2 camas','1 baño',1


In [20]:
def remove_chars(val):
    new_val = re.findall('[0-9.]+',val)
    return "".join(new_val)


airbnbdf.no_of_beds = airbnbdf.no_of_beds.astype(str)
airbnbdf.no_of_baths = airbnbdf.no_of_baths.astype(str)

airbnbdf = airbnbdf.assign(beds=airbnbdf.no_of_beds.apply(remove_chars))
airbnbdf = airbnbdf.assign(bath=airbnbdf.no_of_baths.apply(remove_chars))
airbnbdf.sample(4)

,city,house_title,house_type,price,owner_name,no_of_reviews,rating,occupancy,url,occupancy_lst,no_of_guest,house_size,no_of_beds,no_of_baths,rooms,beds,bath
15,Whistler,Ski in/out Stylish condo with pool/hot tub & v...,Departamento entero,1289,Annika,203,5.0,"['4 huéspedes', '1 recámara', '2 camas', '1 ba...",https://www.airbnb.mx/rooms/4249398?location=W...,"[['4 huéspedes', '1 recámara', '2 camas', '...",4,'1 recámara','2 camas','1 baño',1,2,1
11,Whistler,˙Studio Condo Whistler Village 207,Departamento entero,1520,Bill,286,5.0,"['4 huéspedes', 'estudio', '2 camas', '1 baño'...",https://www.airbnb.mx/rooms/16041781?location=...,"[['4 huéspedes', 'estudio', '2 camas', '1 b...",4,'estudio','2 camas','1 baño',0.5,2,1
16,Whistler,Minimalist Vibe with Amazing Mountain Views,Departamento con servicios completo,1868,Victoria And Christian,337,5.0,"['4 huéspedes', '1 recámara', '1 cama', '1 bañ...",https://www.airbnb.mx/rooms/13283018?location=...,"[['4 huéspedes', '1 recámara', '1 cama', '1...",4,'1 recámara','1 cama','1 baño',1,1,1
8,Whistler,Modern Mountain Studio in the Village,Departamento con servicios completo,1868,Anya,134,5.0,"['2 huéspedes', '1 recámara', '1 cama', '1 bañ...",https://www.airbnb.mx/rooms/14438965?location=...,"[['2 huéspedes', '1 recámara', '1 cama', '1...",2,'1 recámara','1 cama','1 baño',1,1,1


In [21]:
(airbnbdf == "none").any() #check for any none values

city             False
house_title      False
house_type       False
price            False
owner_name       False
no_of_reviews    False
rating           False
occupancy        False
url              False
occupancy_lst    False
no_of_guest      False
house_size       False
no_of_beds       False
no_of_baths      False
rooms            False
beds             False
bath             False
dtype: bool

In [22]:
(airbnbdf.isnull()).any()

city             False
house_title      False
house_type       False
price            False
owner_name       False
no_of_reviews    False
rating           False
occupancy        False
url              False
occupancy_lst    False
no_of_guest      False
house_size       False
no_of_beds       False
no_of_baths      False
rooms            False
beds             False
bath             False
dtype: bool

In [23]:
(airbnbdf.isna()).any()

city             False
house_title      False
house_type       False
price            False
owner_name       False
no_of_reviews    False
rating           False
occupancy        False
url              False
occupancy_lst    False
no_of_guest      False
house_size       False
no_of_beds       False
no_of_baths      False
rooms            False
beds             False
bath             False
dtype: bool

In [24]:
print(len(airbnbdf))
airbnbdf.isna().sum().sum() #check num of Nans

18


0

In [25]:
airbnbdf.dropna() #check 

,city,house_title,house_type,price,owner_name,no_of_reviews,rating,occupancy,url,occupancy_lst,no_of_guest,house_size,no_of_beds,no_of_baths,rooms,beds,bath
0,Whistler,˙Studio Condo Whistler Village 223,Departamento entero,1086,Bill,398,5.0,"['2 huéspedes', 'estudio', '1 cama', '1 baño',...",https://www.airbnb.mx/rooms/12071124?location=...,"[['2 huéspedes', 'estudio', '1 cama', '1 ba...",2,'estudio','1 cama','1 baño',0.5,1,1
1,Whistler,Studio Condo Upper Village Tranquility,Departamento entero,1158,Bill,478,5.0,"['4 huéspedes', 'estudio', '2 camas', '1 baño'...",https://www.airbnb.mx/rooms/7309668?location=W...,"[['4 huéspedes', 'estudio', '2 camas', '1 b...",4,'estudio','2 camas','1 baño',0.5,2,1
2,Whistler,˯Studio Condo Whistler Village 422,Departamento entero,1158,Bill,,5.0,"['3 huéspedes', 'estudio', '2 camas', '1 baño'...",https://www.airbnb.mx/rooms/12310742?location=...,"[['3 huéspedes', 'estudio', '2 camas', '1 b...",3,'estudio','2 camas','1 baño',0.5,2,1
3,Whistler,Peak View Penthouse,Loft entero,1868,Jane & James,,5.0,"['4 huéspedes', '1 recámara', '1 cama', '1 bañ...",https://www.airbnb.mx/rooms/13348663?location=...,"[['4 huéspedes', '1 recámara', '1 cama', '1...",4,'1 recámara','1 cama','1 baño',1,1,1
4,Whistler,1 BR Townhome with Private Hot Tub - FREE PARKING,Casa adosada completa,1665,Ian,312,4.5,"['4 huéspedes', '1 recámara', '2 camas', '1 ba...",https://www.airbnb.mx/rooms/2520507?location=W...,"[['4 huéspedes', '1 recámara', '2 camas', '...",4,'1 recámara','2 camas','1 baño',1,2,1
5,Whistler,Whistler Creekside Villa - single nights now o...,Departamento entero,1144,Beth,338,4.5,"['2 huéspedes', '1 recámara', '1 cama', '1 bañ...",https://www.airbnb.mx/rooms/1295382?location=W...,"[['2 huéspedes', '1 recámara', '1 cama', '1...",2,'1 recámara','1 cama','1 baño',1,1,1
6,Whistler,Newly updated condo at Blackcomb base,Condominio completo,1448,Caroline,122,4.5,"['4 huéspedes', '1 recámara', '1 cama', '1 bañ...",https://www.airbnb.mx/rooms/13126268?location=...,"[['4 huéspedes', '1 recámara', '1 cama', '1...",4,'1 recámara','1 cama','1 baño',1,1,1
7,Whistler,·Hotel Room Upper Village Tranquility,Alojamiento entero,1376,Bill,517,5.0,"['2 huéspedes', '1 cama', '1 baño', '\U000f000...",https://www.airbnb.mx/rooms/7310283?location=W...,"[['2 huéspedes', '1 cama', '1 baño', '\U000...",2,'1 cama','1 baño','\U000f0004\n5056 evaluaciones',1,1,00000045056
8,Whistler,Modern Mountain Studio in the Village,Departamento con servicios completo,1868,Anya,134,5.0,"['2 huéspedes', '1 recámara', '1 cama', '1 bañ...",https://www.airbnb.mx/rooms/14438965?location=...,"[['2 huéspedes', '1 recámara', '1 cama', '1...",2,'1 recámara','1 cama','1 baño',1,1,1
9,Whistler,ˑEight People Upper Village Tranquility,Condominio completo,1231,Bill,319,5.0,"['8 huéspedes', '1 recámara', '4 camas', '2 ba...",https://www.airbnb.mx/rooms/13335461?location=...,"[['8 huéspedes', '1 recámara', '4 camas', '...",8,'1 recámara','4 camas','2 baños',1,4,2


In [26]:
airbnbdfClean = airbnbdf.dropna() #drop all NaN's

In [27]:
print(len(airbnbdfClean))
airbnbdfClean.isna().sum().sum() #check num of Nans

18


0

In [28]:
airbnbdfClean["price"].replace("",0, inplace=True)
airbnbdfClean["no_of_guest"].replace("",0, inplace=True)
airbnbdfClean["rooms"].replace("",0, inplace=True)
airbnbdfClean["beds"].replace("",0, inplace=True)
airbnbdfClean["bath"].replace("",0, inplace=True)

In [30]:
#convert price to float
airbnbdfClean.price=airbnbdfClean.price.astype(float)

#convert no_of_reviews to float
#airbnbdfClean.no_of_reviews=airbnbdfClean.no_of_reviews.astype(int)

#convert no_of_guest to float
airbnbdfClean.no_of_guest=airbnbdfClean.no_of_guest.astype(float)

#convert house to float
airbnbdfClean.rooms=airbnbdfClean.rooms.astype(float)

#convert house to float
airbnbdfClean.beds=airbnbdfClean.beds.astype(float)

#convert house to float
airbnbdfClean.bath=airbnbdfClean.bath.astype(float)

In [31]:
del airbnbdfClean["occupancy"]
del airbnbdfClean["occupancy_lst"]

In [32]:
airbnbdfClean.sample(5)

,city,house_title,house_type,price,owner_name,no_of_reviews,rating,url,no_of_guest,house_size,no_of_beds,no_of_baths,rooms,beds,bath
17,Whistler,Cozy Whistler Village Studio,Condominio completo,1433.0,Tim And Andrea,312,5.0,https://www.airbnb.mx/rooms/4269069?location=W...,4.0,'estudio','2 camas','1 baño',0.5,2.0,1.0
4,Whistler,1 BR Townhome with Private Hot Tub - FREE PARKING,Casa adosada completa,1665.0,Ian,312,4.5,https://www.airbnb.mx/rooms/2520507?location=W...,4.0,'1 recámara','2 camas','1 baño',1.0,2.0,1.0
8,Whistler,Modern Mountain Studio in the Village,Departamento con servicios completo,1868.0,Anya,134,5.0,https://www.airbnb.mx/rooms/14438965?location=...,2.0,'1 recámara','1 cama','1 baño',1.0,1.0,1.0
15,Whistler,Ski in/out Stylish condo with pool/hot tub & v...,Departamento entero,1289.0,Annika,203,5.0,https://www.airbnb.mx/rooms/4249398?location=W...,4.0,'1 recámara','2 camas','1 baño',1.0,2.0,1.0
2,Whistler,˯Studio Condo Whistler Village 422,Departamento entero,1158.0,Bill,,5.0,https://www.airbnb.mx/rooms/12310742?location=...,3.0,'estudio','2 camas','1 baño',0.5,2.0,1.0


In [37]:
airbnbdfClean.to_csv('airbnbdfClean'+filename+'.csv', index=False, encoding="utf-8")
airbnbdfClean.to_excel('airbnbdfClean'+filename+'.xlsx', index=False, encoding="utf-8")